<a href="https://colab.research.google.com/github/rhaulmendonca/Acidentes-por-Aeronave/blob/main/Sistemas%20de%20Apoio%20%C3%A0%20Decis%C3%A3o/CENIPA_Acidentes_por_aeronave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
pip install adjustText

In [5]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [6]:
pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [7]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
import datetime
import calendar
from matplotlib.ticker import PercentFormatter
import geopandas as gpd
import re
import adjustText as aT
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn import metrics
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from catboost.utils import get_confusion_matrix
from catboost import cv
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

file_path = '/content/drive/MyDrive/Colab Notebooks/Sistemas de Apoio a Decisão/126 - CENIPA - Acidentes por Aeronave.xlsx'
df = pd.read_excel(file_path)

In [8]:
num_rows, num_cols = df.shape
print("Número de linhas:", num_rows)
print("Número de colunas:", num_cols)


Número de linhas: 10397
Número de colunas: 55


In [9]:
import os

# Obtendo o tamanho do arquivo em KB
file_size_kb = os.path.getsize(file_path) / 1024
print("Tamanho do arquivo (KB):", file_size_kb)


Tamanho do arquivo (KB): 2525.181640625


In [10]:
df.info()  # Exibe informações sobre cada coluna e tipo de dado


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10397 entries, 0 to 10396
Data columns (total 55 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Código da ocorrência               10397 non-null  int64         
 1   Classificação da ocorrência        10397 non-null  object        
 2   Latitude da ocorrência             10397 non-null  float64       
 3   Longitude da ocorrência            10397 non-null  float64       
 4   Código IBGE do município           10397 non-null  int64         
 5   Nome do município                  10397 non-null  object        
 6   Código IBGE da microrregião        10397 non-null  int64         
 7   Nome da microrregiao               10397 non-null  object        
 8   Código IBGE da mesoregião          10397 non-null  int64         
 9   Nome da mesoregião                 10397 non-null  object        
 10  Código IBGE da UF                 

In [11]:
df.head()


,Código da ocorrência,Classificação da ocorrência,Latitude da ocorrência,Longitude da ocorrência,Código IBGE do município,Nome do município,Código IBGE da microrregião,Nome da microrregiao,Código IBGE da mesoregião,Nome da mesoregião,...,Categoria de registro,Segmento de registro,Aeródromo de origem,Aeródromo de destino,Fase de operação,Nível de dano da aeronave,Total de fatalidades,Tipo de ocorrência,Categoria do tipo de ocorrência,Taxonomia do tipo ICAO
0,78640,ACIDENTE,-16.762062,-49.438403,5200050,Abadia de Goiás,52010,Goiânia,5203,Centro Goiano,...,ULTRALEVE,EXPERIMENTAL,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,SUBSTANCIAL,0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
1,46633,ACIDENTE,-16.762062,-49.438403,5200050,Abadia de Goiás,52010,Goiânia,5203,Centro Goiano,...,AVIÃO,EXPERIMENTAL,FORA DE AERODROMO,FORA DE AERODROMO,MANOBRA,SUBSTANCIAL,0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
2,80154,ACIDENTE,-1.962741,-48.196593,1500206,Acará,15012,Tomé-Açu,1504,Nordeste Paraense,...,AVIÃO,PARTICULAR,NÃO IDENTIFICADO,NÃO IDENTIFICADO,Sem Informação,LEVE,0,INDETERMINADO,INDETERMINADO,UNK
3,38855,ACIDENTE,-31.867518,-54.163801,4300034,Aceguá,43031,Campanha Meridional,4306,Sudoeste Rio-grandense,...,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,SUBSTANCIAL,1,PERDA DE CONTROLE EM VOO,PERDA DE CONTROLE EM VOO,LOC-I
4,50831,ACIDENTE,-17.400018,-50.379307,5200134,Acreúna,52014,Vale do Rio dos Bois,5205,Sul Goiano,...,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,DESTRUÍDA,4,PERDA DE CONTROLE EM VOO,PERDA DE CONTROLE EM VOO,LOC-I


In [12]:
df.dtypes


,0
Código da ocorrência,int64
Classificação da ocorrência,object
Latitude da ocorrência,float64
Longitude da ocorrência,float64
Código IBGE do município,int64
Nome do município,object
Código IBGE da microrregião,int64
Nome da microrregiao,object
Código IBGE da mesoregião,int64
Nome da mesoregião,object


In [13]:
df.describe()

,Código da ocorrência,Latitude da ocorrência,Longitude da ocorrência,Código IBGE do município,Código IBGE da microrregião,Código IBGE da mesoregião,Código IBGE da UF,Código IBGE da região,Dia da ocorrência,Ano da ocorrência,Data e hora da ocorrência,Dia de divulgação da publicação,Total de recomendações,Total de aeronaves envolvidas,PMD aeronave,Categoria PMD aeronave,Quantidade de assentos,Ano de fabricação,Total de fatalidades
count,10397.000000,10397.000000,10397.000000,1.039700e+04,10397.000000,10397.000000,10397.000000,10397.000000,10397,10397.000000,10397,10397,10397.000000,10397.000000,10397.000000,10397.000000,10397.000000,10397.000000,10397.000000
mean,62441.940848,-18.392234,-48.115042,3.461259e+06,34487.253438,3452.180244,34.587670,3.127729,2016-04-27 09:13:43.737616640,2015.814947,2016-04-28 00:07:00.742041088,1929-08-19 04:23:34.100221184,0.361354,1.022891,21161.756372,21161.756372,34.723670,1857.477253,0.145619
min,28256.000000,-884.917000,-72.787966,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2007-01-01 00:00:00,2007.000000,2007-01-01 00:00:00,1900-01-02 00:00:00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,46306.000000,-23.404066,-51.227035,3.106200e+06,31018.000000,3105.000000,31.000000,3.000000,2011-10-25 00:00:00,2011.000000,2011-10-25 14:40:00,1900-01-02 00:00:00,0.000000,1.000000,1270.000000,1270.000000,2.000000,1977.000000,0.000000
50%,65098.000000,-21.178000,-47.880612,3.516853e+06,35032.000000,3507.000000,35.000000,3.000000,2016-03-07 00:00:00,2016.000000,2016-03-07 14:57:00,1900-01-02 00:00:00,0.000000,1.000000,2155.000000,2155.000000,6.000000,1992.000000,0.000000
75%,80090.000000,-13.553388,-44.210987,4.204202e+06,42008.000000,4202.000000,42.000000,4.000000,2021-05-09 00:00:00,2021.000000,2021-05-09 20:00:00,2009-03-09 00:00:00,0.000000,1.000000,23000.000000,23000.000000,13.000000,2008.000000,0.000000
max,83097.000000,4.595018,0.000000,5.300108e+06,53001.000000,5301.000000,53.000000,5.000000,2023-12-31 00:00:00,2023.000000,2023-12-31 22:30:00,2023-12-29 00:00:00,83.000000,4.000000,397000.000000,397000.000000,659.000000,9999.000000,199.000000
std,17456.408952,14.369120,6.962138,1.116616e+06,11160.267604,1116.051957,10.981005,1.191398,NaN,5.234625,NaN,NaN,1.586162,0.162510,39945.063114,39945.063114,64.004453,506.471969,2.066319


In [19]:
def get_duplicates(df):
 coluna = [column for column in df.columns if 'Código da ocorrência' in column]
 coluna = coluna[0]
 duplicados = df.loc[df[coluna].duplicated()][coluna].values
 df_duplicados = df[df[coluna].isin(duplicados)].sort_values(by=coluna)
 return df_duplicados

In [21]:
get_duplicates(df)

,Código da ocorrência,Classificação da ocorrência,Latitude da ocorrência,Longitude da ocorrência,Código IBGE do município,Nome do município,Código IBGE da microrregião,Nome da microrregiao,Código IBGE da mesoregião,Nome da mesoregião,...,Categoria de registro,Segmento de registro,Aeródromo de origem,Aeródromo de destino,Fase de operação,Nível de dano da aeronave,Total de fatalidades,Tipo de ocorrência,Categoria do tipo de ocorrência,Taxonomia do tipo ICAO
9905,31255,ACIDENTE,-24.166843,-54.096779,4127403,Terra Roxa,41022,Toledo,4106,Oeste Paranaense,...,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,DESTRUÍDA,0,POUSO LONGO,EXCURSÃO DE PISTA | POUSO LONGO,RE
9907,31255,ACIDENTE,-24.166843,-54.096779,4127403,Terra Roxa,41022,Toledo,4106,Oeste Paranaense,...,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,DESTRUÍDA,0,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G
2932,31875,INCIDENTE GRAVE,-15.600666,-56.097856,5103403,Cuiabá,51017,Cuiabá,5104,Centro-Sul Mato-grossense,...,AVIÃO,INSTRUÇÃO,MARECHAL RONDON,ESTÂNCIA SANTA RITA,DECOLAGEM,LEVE,0,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G
2928,31875,INCIDENTE GRAVE,-15.600666,-56.097856,5103403,Cuiabá,51017,Cuiabá,5104,Centro-Sul Mato-grossense,...,AVIÃO,INSTRUÇÃO,MARECHAL RONDON,ESTÂNCIA SANTA RITA,DECOLAGEM,LEVE,0,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
542,32916,ACIDENTE,-2.748361,-42.826727,2101707,Barreirinhas,21004,Lençóis Maranhenses,2101,Norte Maranhense,...,AVIÃO,TÁXI AÉREO,RANCHO SUMIDOR,RANCHO SUMIDOR,CORRIDA APÓS POUSO,SUBSTANCIAL,0,COLISÃO COM OBSTÁCULOS NO SOLO,COLISÃO NO SOLO | COLISÃO COM OBSTÁCULOS NO SOLO,GCOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,82561,INCIDENTE GRAVE,-23.153000,-47.058000,3524006,Itupeva,35047,Jundiaí,3512,Macro Metropolitana Paulista,...,ULTRALEVE,EXPERIMENTAL,MURARO - SP,MURARO - SP,CRUZEIRO,LEVE,0,FALHA OU MAU FUNCIONAMENTO DO MOTOR,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP
8902,82600,INCIDENTE,-25.532994,-49.183860,4125506,São José dos Pinhais,41037,Curitiba,4110,Metropolitana de Curitiba,...,AVIÃO,Sem Informação,HERCÍLIO LUZ,GOVERNADOR JOSÉ RICHA,CRUZEIRO,NENHUM,0,FALHA OU MAU FUNCIONAMENTO DO MOTOR,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP
8900,82600,INCIDENTE,-25.532994,-49.183860,4125506,São José dos Pinhais,41037,Curitiba,4110,Metropolitana de Curitiba,...,AVIÃO,Sem Informação,HERCÍLIO LUZ,GOVERNADOR JOSÉ RICHA,CRUZEIRO,NENHUM,0,FOGO/FUMAÇA (SEM IMPACTO),FOGO/FUMAÇA (SEM IMPACTO),F-NI
8959,82874,ACIDENTE,-15.628746,-58.180367,5107107,São José dos Quatro Marcos,51014,Jauru,5103,Sudoeste Mato-grossense,...,AVIÃO,AGRÍCOLA,FORA DE AERODROMO,FORA DE AERODROMO,MANOBRA,SUBSTANCIAL,1,PERDA DE CONTROLE EM VOO,PERDA DE CONTROLE EM VOO,LOC-I


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-25-dbe03998f7b6>, line 1)